In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

In [2]:
path = "/kaggle/input/england-rufc-results-2020-23/EnglandRUFCML.xlsx"

In [3]:
df = pd.read_excel(path)

In [4]:
df.head()

,Margin,Opponent,Location,WrldRnk,OppWrldRnk,ResPrevOpp,ResPrevFixt,Year
0,-7,France,A,3,7,-20,36,2020
1,7,Scotland,A,3,9,-7,0,2020
2,12,Ireland,H,3,4,7,42,2020
3,3,Wales,H,3,6,12,-8,2020
4,29,Italy,A,3,7,3,37,2020


In [5]:
df = df.replace('A', 0)
df = df.replace('H', 1)

In [6]:
df.OppWrldRnk = pd.qcut(df.OppWrldRnk,6)

In [7]:
df['OppWrldRnk'] = df['OppWrldRnk'].apply(lambda x: x.mid)

In [8]:
df.OppWrldRnk.groupby(df.OppWrldRnk).count()

OppWrldRnk
1.9995     9
4.5       11
6.5        6
8.0        8
10.5       5
17.5       7
Name: OppWrldRnk, dtype: int64

In [9]:
df.ResPrevOpp = pd.qcut(df.ResPrevOpp,5)

In [10]:
df['ResPrevOpp'] = df['ResPrevOpp'].apply(lambda x: x.mid)

In [11]:
df.ResPrevOpp.groupby(df.ResPrevOpp).count()

ResPrevOpp
-25.5005    10
-3.5        10
4.5          8
12.5         9
44.0         9
Name: ResPrevOpp, dtype: int64

In [12]:
df.ResPrevFixt = pd.qcut(df.ResPrevFixt,5)

In [13]:
df['ResPrevFixt'] = df['ResPrevFixt'].apply(lambda x: x.mid)

In [14]:
df.ResPrevFixt.groupby(df.ResPrevFixt).count()

ResPrevFixt
-13.5005    10
-2.0         9
8.0          9
23.0        10
52.0         8
Name: ResPrevFixt, dtype: int64

In [15]:
y=df.Margin
feature_columns = ["Location", "WrldRnk", "OppWrldRnk", "ResPrevOpp", "ResPrevFixt"]
X = df[feature_columns]

In [16]:
rugby_model = DecisionTreeRegressor()
rugby_model.fit(X,y)

DecisionTreeRegressor()

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
train_X, val_X, train_y, val_y = train_test_split(X,y, random_state=1)

In [19]:
rugby_model=DecisionTreeRegressor(random_state=1)
rugby_model.fit(train_X, train_y)

DecisionTreeRegressor(random_state=1)

In [20]:
val_predictions = rugby_model.predict(val_X)
print(val_predictions)

[ 10.   1.  29.   7.   1.  33. -11.  -6. -11.  17. -11.   8.]


In [21]:
from sklearn.metrics import mean_absolute_error

In [22]:
val_mae = mean_absolute_error(val_y, val_predictions)
print(val_mae)

18.666666666666668


In [23]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return mae

In [24]:
candidate_max_leaf_nodes = [2,3,4,5,6,7,8,9,10]

for vals in candidate_max_leaf_nodes:
    my_mae = get_mae(vals, train_X, val_X, train_y, val_y)
    print("Max lead nodes: %d \t\t Mean Absolute Error: %d" %(vals, my_mae))

Max lead nodes: 2 		 Mean Absolute Error: 16
Max lead nodes: 3 		 Mean Absolute Error: 16
Max lead nodes: 4 		 Mean Absolute Error: 17
Max lead nodes: 5 		 Mean Absolute Error: 17
Max lead nodes: 6 		 Mean Absolute Error: 16
Max lead nodes: 7 		 Mean Absolute Error: 16
Max lead nodes: 8 		 Mean Absolute Error: 16
Max lead nodes: 9 		 Mean Absolute Error: 16
Max lead nodes: 10 		 Mean Absolute Error: 17


In [25]:
final_model = DecisionTreeRegressor(max_leaf_nodes=9, random_state=0)
final_model.fit(X,y)

DecisionTreeRegressor(max_leaf_nodes=9, random_state=0)

Predicting the result of the England vs Argentina opening game...

In [26]:
new_data = pd.DataFrame({'Location': [0],
                         'WrldRnk': [5],
                        'OppWrldRnk': [6],
                        'ResPrevOpp': [12.5],
                        'ResPrevFixt': [-2]})

In [27]:
final_model.predict(new_data)

array([1.4])

Prediction: England by 1.4 points